In [1]:
import calendar
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.max_row',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 6, 25), datetime.date(2023, 6, 24))

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
#yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-06-25
yesterday: 2023-06-23 00:00:00


In [3]:
yesterday = yesterday.date()
a_year_ago = yesterday - timedelta(days=365)
yesterday, a_year_ago

(datetime.date(2023, 6, 23), datetime.date(2022, 6, 23))

### Restart and Run All Cells

### Weekly process or when stataus changes.

In [4]:
pd.read_sql_query('SELECT COUNT(*) AS records FROM sales', conlite)

,records
0,3513


In [5]:
sqlDel = """DELETE FROM sales"""
rp = conlite.execute(sqlDel)
rp.rowcount

4030

In [6]:
sql = """
SELECT name
FROM orders 
ORDER BY name
"""
df = pd.read_sql(sql, conlite)

names = df['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AH', 'AWC', 'BANPU', 'BBL', 'BCH', 'BDMS', 'BEM', 'BH', 'CPNREIT', 'DIF', 'EA', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KKP', 'KTB', 'KTC', 'LANNA', 'MCS', 'NER', 'ORI', 'PTG', 'PTTEP', 'PTTGC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TASCO', 'TFFIF', 'TMT', 'TTB'"

In [7]:
len(in_p.split(','))

35

### Get past one year data

In [8]:
sql = """
SELECT name, date, price, qty, maxp, minp 
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (a_year_ago, in_p)
print(sql)


SELECT name, date, price, qty, maxp, minp 
FROM price 
WHERE date >= '2022-06-23' AND name IN ('AH', 'AWC', 'BANPU', 'BBL', 'BCH', 'BDMS', 'BEM', 'BH', 'CPNREIT', 'DIF', 'EA', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KKP', 'KTB', 'KTC', 'LANNA', 'MCS', 'NER', 'ORI', 'PTG', 'PTTEP', 'PTTGC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TASCO', 'TFFIF', 'TMT', 'TTB') 
ORDER BY name, date


In [9]:
df = pd.read_sql(sql, const)
df.head()

,name,date,price,qty,maxp,minp
0,AH,2022-06-23,21.6,477020,22.1,21.6
1,AH,2022-06-24,21.6,546584,21.7,21.4
2,AH,2022-06-27,22.1,365071,22.2,21.7
3,AH,2022-06-28,22.6,257044,22.7,22.0
4,AH,2022-06-29,22.5,233860,22.5,22.1


In [10]:
data_path = "../data/"
file_name = "Yearly-Price-by-Name.csv"
output_file = data_path + file_name

df.set_index("name", inplace=True)
df.to_csv(output_file, header=None)

### Create monitors from orders

In [11]:
sql = """
SELECT name,trade 
FROM orders 
WHERE name IN (%s)
ORDER BY name
"""
sql = sql % in_p
orders = pd.read_sql(sql, conlite)
orders.set_index(['name'],inplace=True)
orders

,trade
name,
AH,S
AWC,S
BANPU,B
BBL,B
BCH,S
BDMS,B
BEM,B
BH,B
CPNREIT,B


In [12]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

orders.to_csv(data_file, header=None)
orders.to_csv(output_file)
orders.to_csv(box_file)

In [13]:
sql = """
SELECT trade, COUNT(*) AS items 
FROM orders
GROUP BY trade
ORDER BY trade
"""
grp = pd.read_sql(sql, conlite)
grp

,trade,items
0,B,25
1,S,10


### For new orders that never have XXX.html & XXX.CSV

In [14]:
file_name = 'price-uploads.csv'
url = data_path + file_name
url

'../data/price-uploads.csv'

In [15]:
uploads = pd.read_csv(url)
uploads.sort_values(['name'],ascending=[True]).shape

(151, 1)

In [16]:
df_merge = pd.merge(orders, uploads, on='name', how='outer', indicator=True)
df_merge.sort_values(['name'],ascending=[True]).shape

(157, 3)

In [17]:
new_prices = df_merge['_merge'] == 'left_only'
df_merge[new_prices]

,name,trade,_merge
0,AH,S,left_only
1,AWC,S,left_only
6,BEM,B,left_only
12,ICHI,S,left_only
20,LANNA,B,left_only
31,TASCO,B,left_only


In [18]:
new_prices = df_merge['_merge'] == 'right_only'
df_merge[new_prices].shape

(122, 3)

In [19]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks.shape

(80, 2)

In [20]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)
stocks.to_csv(one_file)

In [21]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""
buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
BANPU,B
BBL,O
BDMS,O
BEM,O
BH,O
CPNREIT,B
DIF,B
EA,O
FPT,O


In [22]:
buy_candidates.shape[0]

28

In [23]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
AH,I
AWC,I
BCH,I
GVREIT,I
ICHI,I
IVL,S
JASIF,S
KTC,I
PTTGC,S


In [24]:
sell_candidates.shape[0]

10